# Funções para encontrar o cm_px

In [1]:
#DETERMINA A QUANTIDADE DE CIRCULOS QUE SERAM PECORRIDOS
def contar_circulos(tamanho_quadrado, raio_circulo):
    
    quantidade_circulos = tamanho_quadrado // (raio_circulo * 2)
    
    return quantidade_circulos

In [2]:
def obter_cm_px(img, numero_raios, tamanho, x, y, pontosJson):
    
    """
    Função: Obter o valor do cm por pixel, e marcar na imagem a região de referência.
    Paramêtros:
        @img: imagem para visualização
        @numero_raios: quantidade de raios desejados
        @tamanho: tamanhos dos raios
        @x e @y: coordenada do ponto de referência
        @pontosJson: matriz com as distância de cada coordenada
    Retorno: Valor do cm por pixel de cada manga   
    """
    
    cores = [255, 0, 0]
    increm = 1
    raioFinal = 0
    
    somaCM = 0
    somaPX = 0
    
    while(increm <= numero_raios):
        
        raioFinal = increm * tamanho
        
        x_1 = x - raioFinal
        x1  = x + raioFinal
        
        distancias = lidar.measureDistance((x_1, y), (x1, y), pontosJson)
        
        cv2.circle(img, 
                   (int(x * 7.5 / REDUCTION), 
                    int(y * 7.5 / REDUCTION)), 
                    int(raioFinal * 7.5 / REDUCTION), 
                    cores)
        
        cv2.circle(img, 
                   (int(x_1 * 7.5 / REDUCTION), 
                    int(y   * 7.5 / REDUCTION)), 
                    int(2), 
                    (0, 0, 255))
                
        cv2.circle(img, 
                   (int(x1 * 7.5 / REDUCTION), 
                    int(y  * 7.5 / REDUCTION)), 
                    int(2), 
                    (0, 0, 255))

        somaCM += distancias
        somaPX += (raioFinal * 2)
        
        increm += 1
        
    cm_px = (somaCM / somaPX) * 100
    
    return cm_px

In [3]:
def fator_pixel(img, x, y, pontosJson, NUMERO_RAIOS, TAMANHO_RAIOS):
    
    #global NUMERO_RAIOS, TAMANHO_RAIOS
    
    f_pixel = 0
        
    click_x_r = x
    click_y_r = y

    cv2.circle(img, (int(click_x_r * 7.5 / REDUCTION), int(click_y_r * 7.5 / REDUCTION)), 2, (255, 0, 0), -1)

    cm_px = obter_cm_px(img, NUMERO_RAIOS, TAMANHO_RAIOS, click_x_r, click_y_r, pontosJson)

    cv2.imshow("Imagem", img)

    return cm_px

In [4]:
def calcula_distancias_entre_pontos(ponto1, ponto2, pontosJson, cm_px):
    
    distancia_p1_p2 = (( (ponto1[0] - ponto2[0]) ** 2) + ((ponto1[1] - ponto2[1]) ** 2)) ** 0.5
    
    distancia_final = distancia_p1_p2 * cm_px
    
    return distancia_final

In [5]:
def encontra_ponto_2(ponto1, altura, cm_px):
    
    ponto2_y = int((ponto1[1] - (REDUCTION * altura / cm_px)))

    ponto2_x = ponto1[0]

    ponto2 = (ponto2_x, ponto2_y)
    
    return ponto2

# Realiza marcações

In [6]:
def marca_area(img, coordenadaInicial, coordenadaFinal, cor, opacidade, label):
    
    cv2.putText(img, label, 
                (coordenadaInicial[0], coordenadaInicial[1] - 5), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.5, 
                cor, 
                2, 
                1)
    
    cv2.rectangle(img, coordenadaInicial, coordenadaFinal, cor, int(opacidade))

In [7]:
def label_image(event, x, y, flags, param):
    
    global xt, yt, xb, yb, botao, img, controle, coordenadas, contador, array_fator_cm_px
    global TIPO_BASE, jsonPontos, REDUCTION, ALTURA_DESEJADA
    
    cor = 0
    label = ""
    
    COR_MANGA =     (0, 0, 255)
    COR_PEDUNCULO = (0, 255, 0)
    COR_PONTO =     (255, 0, 0)
    
    if(controle == 1):
        
        cor = COR_MANGA
        label = "mango_" + str(contador)
        
    elif(controle == 2):
        
        cor = COR_PEDUNCULO
        label = "penducle_" + str(contador)
        
    elif(controle == 3 and TIPO_BASE == "3D"):

        cor = COR_PONTO
        label = "point_" + str(contador)

    if event == cv2.EVENT_LBUTTONDOWN:
        
        botao = True
        xt, yt = x, y
        controle += 1
        
        if((TIPO_BASE == "2D" and controle == 3) or (TIPO_BASE == "3D" and controle == 4)):
            
            controle = 1
            contador += 1
        
    elif event == cv2.EVENT_LBUTTONUP:
        
        botao = False
        xb, yb = x, y
        
        xt, xb = min(xt, xb), max(xt, xb)
        yt, yb = min(yt, yb), max(yt, yb)
        
        if(controle == 1 and TIPO_BASE == "3D"):
            
            #Encontra centro da manga x
            centro_manga_x = int(((xt + xb) / 2) / 7.5 * REDUCTION)
            centro_manga_x_ponto_2 = int((xt + xb) / 2)
            
            #Encontra centro da manga y
            centro_manga_y = int(((yt + yb) / 2) / 7.5 * REDUCTION)
            
            #FAZ REDUÇÃO NA CAIXA DA MANGA PARA ENCONTRAR O CM_PX
            largura_reduzida = int(((xb - xt) * 0.2))
            altura_reduzida  = int(((yb - yt) * 0.2))

            xt_r = largura_reduzida + xt
            yt_r = altura_reduzida  + yt

            xb_r = xb - largura_reduzida
            yb_r = yb - altura_reduzida 

            xt_r = int(xt_r)
            xb_r = int(xb_r)
            yt_r = int(yt_r)
            yb_r = int(yb_r)
            
            cv2.rectangle(imagem, (xt_r, yt_r), (xb_r, yb_r), (255, 255, 0))
            
            largura_caixa_reduzida = int((xb_r - xt_r) / 7.5 * REDUCTION)
                                    
            quantidade_raios = contar_circulos(largura_caixa_reduzida, 2)
                        
            #Função para encontrar cm_px            
            fator_cm_px = fator_pixel(imagem, centro_manga_x, centro_manga_y, jsonPontos, quantidade_raios, 2)
            
            array_fator_cm_px.append(fator_cm_px)
            
            print(f'Distância Manga {contador}: {lidar.measureDistanceOnePoint((centro_manga_x, centro_manga_y), jsonPontos)}')
            
            #Função para encontrar a coordenada da altura desejada em cm
            coord_altura_desejada = encontra_ponto_2((centro_manga_x_ponto_2, yt), ALTURA_DESEJADA, fator_cm_px)
            cv2.circle(imagem, (centro_manga_x_ponto_2, yt), 2, (255, 0, 0), -1) #Marca topo da caixa
            
            #Marca area do ponto
            cv2.circle(imagem, coord_altura_desejada, 2, (255, 0, 0), -1)
            cv2.rectangle(imagem, (coord_altura_desejada[0] - 15, coord_altura_desejada[1] - 15), 
                         (coord_altura_desejada[0] + 15, coord_altura_desejada[1] + 15), (255, 0, 0))
         
        if(TIPO_BASE == "2D" and controle < 3):
        
            marca_area(imagem, (xt, yt), (xb, yb), cor, 2, label)
            coordenadas.append(((xt, yt), (xb, yb)))
            
        elif(TIPO_BASE == "3D" and controle < 4):
        
            marca_area(imagem, (xt, yt), (xb, yb), cor, 2, label)
            
            if(controle == 3):
                
                coordenadas.append(((xt, yt), (xb, yb)))
                
            else:
                
                coordenadas.append(((xt, yt), (xb, yb)))

    if botao:
        
        image_temp = imagem.copy()
        
        if((TIPO_BASE == "2D" and controle < 3) or (TIPO_BASE == "3D" and controle < 4)):
            
            marca_area(image_temp, (xt, yt), (x, y), cor, 2, label)
            
        cv2.imshow('Imagem', image_temp)
        
    else:
        
        cv2.imshow('Imagem', imagem)

In [8]:
def salva_json(nome_imagem, arquivos_json, coordenadas, array_fator_cm_px):
    
    dividi_lista = 0
        
    with open(arquivo_json, 'r+') as file:
            
            file_content = file.read()
            
            if not file_content:
                
                data = {}
                
            else:
                
                file.seek(0)
                data = json.load(file)
                    
            remove_chaves = [cha for cha in data.keys() if cha.startswith(("MangoCoordinates_", 
                                                                            "PeduncleCoordinates_", 
                                                                            "PointCoordinates_",
                                                                            "Factor_"))]
            
            for chaves in remove_chaves:
                
                del data[chaves]
            
            label_json = ""
            
            tamanho_coordenadas = len(coordenadas)
            
            if(TIPO_BASE == "2D"):
                             
                quantidade_manga = tamanho_coordenadas // 2
                dividi_lista = 2
                
            else:
                
                quantidade_manga = tamanho_coordenadas // 3
                dividi_lista = 3
            
            data['InformationImage'] = {
                
                'name_image': nome_imagem,
                
                'reduction_value': REDUCTION,
                
                'mango_quantity': quantidade_manga
            }
            
            file.seek(0)
            json.dump(data, file, indent=4)
                
            for qm in range(quantidade_manga):
                
                for coord in range(dividi_lista):

                    if(coord == 0):

                        label_json = "MangoCoordinates_" + str(qm)

                    elif(coord == 1):

                        label_json = "PeduncleCoordinates_" + str(qm)

                    else:

                        if(TIPO_BASE == "3D"):

                            label_json = "PointCoordinates_" + str(qm)
                            
                    coordenada_atual = coordenadas[0]
                    
                    data[label_json] = {
                        
                        'xt': coordenada_atual[0][0] * REDUCTION,
                        'yt': coordenada_atual[0][1] * REDUCTION,
                        'xb': coordenada_atual[1][0] * REDUCTION,
                        'yb': coordenada_atual[1][1] * REDUCTION
                    }

                    coordenadas.pop(0)
                
                if(TIPO_BASE == "3D"):
                
                    fator_atual = array_fator_cm_px[0]

                    data['Factor_' + str(qm)] = {

                        'value': fator_atual
                    }

                    array_fator_cm_px.pop(0)

            file.seek(0)
            file.truncate()
            json.dump(data, file, indent=4)

In [9]:
def visualiza_marcacoes(CAMINHO_IMAGEM, ARQUIVO_JSON, TIPO_BASE):
    
    global idx
        
    while(True):
                    
        key2 = cv2.waitKey(1)

        if key2 == 27:

            cv2.destroyWindow('Visualizacao')
            
            break
    
        imagem = cv2.imread(CAMINHO_IMAGEM)

        if imagem is None:

            print("Imagem não encontrada!")

            idx += 1
        
            print("--------------------------------------")

        ALTURA, LARGURA, _ = imagem.shape

        with open(ARQUIVO_JSON) as file:

            coordenadas = json.load(file)
            
        information_image = coordenadas.get('InformationImage')
        
        if information_image is None:
            
            print("Não existe informação para vizualização")
            
            idx += 1
        
            print("--------------------------------------")
            
            break

        REDUCAO_IMAGEM = coordenadas['InformationImage']['reduction_value']
        QUANTIDADE_IMAGEM = coordenadas['InformationImage']['mango_quantity']

        LARGURA = int(LARGURA / REDUCAO_IMAGEM)
        ALTURA  = int(ALTURA / REDUCAO_IMAGEM)

        imagem = cv2.resize(imagem, (LARGURA, ALTURA))
        
        for visu in range(QUANTIDADE_IMAGEM):
            
            xt, yt = coordenadas['MangoCoordinates_' + str(visu)]['xt'] / REDUCAO_IMAGEM, coordenadas['MangoCoordinates_' + str(visu)]['yt'] / REDUCAO_IMAGEM
            xb, yb = coordenadas['MangoCoordinates_' + str(visu)]['xb'] / REDUCAO_IMAGEM, coordenadas['MangoCoordinates_' + str(visu)]['yb'] / REDUCAO_IMAGEM

            cv2.rectangle(imagem, (int(xt), int(yt)), (int(xb), int(yb)), (0, 0, 255), 2)
            cv2.putText(imagem, "mango_" + str(visu), (int(xt), int(yt) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, 1)

            xt, yt = coordenadas['PeduncleCoordinates_' + str(visu)]['xt'] / REDUCAO_IMAGEM, coordenadas['PeduncleCoordinates_' + str(visu)]['yt'] / REDUCAO_IMAGEM
            xb, yb = coordenadas['PeduncleCoordinates_' + str(visu)]['xb'] / REDUCAO_IMAGEM, coordenadas['PeduncleCoordinates_' + str(visu)]['yb'] / REDUCAO_IMAGEM

            cv2.rectangle(imagem, (int(xt), int(yt)), (int(xb), int(yb)), (0, 255, 0), 2)
            cv2.putText(imagem, "peduncle_" + str(visu), (int(xt), int(yt) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, 1)

            if(TIPO_BASE == "3D"):

                xt, yt = coordenadas['PointCoordinates_' + str(visu)]['xt'] / REDUCAO_IMAGEM, coordenadas['PointCoordinates_' + str(visu)]['yt'] / REDUCAO_IMAGEM
                xb, yb = coordenadas['PointCoordinates_' + str(visu)]['xb'] / REDUCAO_IMAGEM, coordenadas['PointCoordinates_' + str(visu)]['yb'] / REDUCAO_IMAGEM

                cv2.rectangle(imagem, (int(xt), int(yt)), (int(xb), int(yb)), (255, 0, 0), 2)
                cv2.putText(imagem, "mango_" + str(visu), (int(xt), int(yt) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, 1)

        cv2.imshow("Visualizacao", imagem)    

In [10]:
import cv2
import json
import numpy as np
import os

import Lidar as lidar

botao = False
xt, yt = -1, -1  
xb, yb = -1, -1
coordenadas = [ ]
array_fator_cm_px = [ ]

#######################################################################
PASTA       = "VALID_LIDAR_IMAGES_3D/"
SUBPASTA    = "VL_MANGO_1/"
NOME_IMAGEM = "_MANGO_IMAGE"
NOME_JSON   = "_MANGO_MAP_JSON"

idx = 1
QUANTIDADE_IMAGEM = 6

TIPO_BASE = "3D"

NUMERO_RAIOS = 4
TAMANHO_RAIOS = 2

ALTURA_DESEJADA = 3

REDUCTION = 3
#######################################################################

while(idx <= QUANTIDADE_IMAGEM):
    
    cv2.namedWindow('Imagem')
    cv2.setMouseCallback('Imagem', label_image)
    
    controle = contador = 0
    
    caminho_imagem = PASTA + SUBPASTA + str(idx) + NOME_IMAGEM + ".jpg"
    print(f"Caminho imagem -> {caminho_imagem}")
    
    imagem = cv2.imread(caminho_imagem)
    
    if imagem is None:
        
        print(f"Erro ao carregar a imagem {caminho_imagem}")
        
        idx += 6
        
        continue
        
    cv2.putText(imagem, "Imagem: " + str(idx), 
                (30, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1.9, 
                (0, 0, 255), 
                6, 
                6)
    
    nome_imagem = str(idx) + NOME_IMAGEM + ".jpg"
    
    arquivo_json   = PASTA + SUBPASTA + str(idx) + NOME_JSON + ".json"
    
    if not os.path.exists(arquivo_json):
        
        print(f"Criando json -> {arquivo_json}")
            
        open(arquivo_json, 'w')
        
    print(f"Caminho json   -> {arquivo_json}")
    
    if(TIPO_BASE == "3D"):
    
        jsonPontos = lidar.organizaJSON(arquivo_json)
    
    ALTURA, LARGURA, _ = imagem.shape
    
    LARGURA = int(LARGURA / REDUCTION)
    ALTURA  = int(ALTURA / REDUCTION)
    
    imagem = cv2.resize(imagem, (LARGURA, ALTURA))
    
    cv2.imshow('Imagem', imagem)

    key = cv2.waitKey(0)

    if key == 27:
        
        print("\n\nFinalizado!")
        
        break
        
    if key == ord('p'):
        
        if(len(coordenadas) < 1):
            
            print("Não houve marcacoes!")
        
        else:
        
            salva_json(nome_imagem, arquivo_json, coordenadas, array_fator_cm_px)
        
            xt, yt = -1, -1  
            xb, yb = -1, -1
            coordenadas = [ ]
            array_fator_cm_px = [ ]

            print(f"Imagem -> {nome_imagem} salva")
        
        idx += 1
        
        print("--------------------------------------")
        
    elif key == ord('l'):
        
        xt, yt = -1, -1  
        xb, yb = -1, -1
        coordenadas = [ ]
        
        print(f"Imagem: {caminho_imagem} limpa")
        
        imagem = cv2.imread(caminho_imagem)
        
    elif key == ord('v'):
        
        visualiza_marcacoes(caminho_imagem, arquivo_json, TIPO_BASE)

cv2.destroyAllWindows()

Caminho imagem -> VALID_LIDAR_IMAGES_3D/VL_MANGO_1/1_MANGO_IMAGE.jpg
Caminho json   -> VALID_LIDAR_IMAGES_3D/VL_MANGO_1/1_MANGO_MAP_JSON.json
Distância Manga 0: 20.0927734375
Imagem: VALID_LIDAR_IMAGES_3D/VL_MANGO_1/1_MANGO_IMAGE.jpg limpa
Caminho imagem -> VALID_LIDAR_IMAGES_3D/VL_MANGO_1/1_MANGO_IMAGE.jpg
Caminho json   -> VALID_LIDAR_IMAGES_3D/VL_MANGO_1/1_MANGO_MAP_JSON.json
Distância Manga 0: 20.0927734375
Imagem -> 1_MANGO_IMAGE.jpg salva
--------------------------------------
Caminho imagem -> VALID_LIDAR_IMAGES_3D/VL_MANGO_1/2_MANGO_IMAGE.jpg
Caminho json   -> VALID_LIDAR_IMAGES_3D/VL_MANGO_1/2_MANGO_MAP_JSON.json
Distância Manga 0: 19.921875
Imagem -> 2_MANGO_IMAGE.jpg salva
--------------------------------------
Caminho imagem -> VALID_LIDAR_IMAGES_3D/VL_MANGO_1/3_MANGO_IMAGE.jpg
Caminho json   -> VALID_LIDAR_IMAGES_3D/VL_MANGO_1/3_MANGO_MAP_JSON.json
Distância Manga 0: 19.9951171875
Imagem: VALID_LIDAR_IMAGES_3D/VL_MANGO_1/3_MANGO_IMAGE.jpg limpa
Caminho imagem -> VALID_LIDA